In [1]:
# imports
import os
import pandas as pd
from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType
from lightgbm import LGBMRegressor
from src.pipeline.pipeline_factory import PipelineFactory
from src.features.encoder_utils import load_graph
from src.pipeline.pipeline_transformers import PoincareEmbedding, ColumnKeeper, OpenMLMetaFeatureTransformer, GeneralPurposeEncoderTransformer
from category_encoders.binary import BinaryEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder


train_df = config.load_traindata_for_regression()
pipelineFactory = PipelineFactory()

graph = load_graph(config.ROOT_DIR / "data/external/graphs/encodings_graph.adjlist")


param_grid = {
    "embeddings_transformer__epochs": [500],
    "embeddings_transformer__batch_size": [50, 500, 1000],
    # "embeddings_transformer__size": [2, 100, 150, 200],
    # "embeddings_transformer__alpha": [0.1, 0.2, 0.5],
    "general_transformer__model_encoder" : [BinaryEncoder(), OneHotEncoder(), OrdinalEncoder(), TargetEncoder()],
    "general_transformer__tuning_encoder" : [BinaryEncoder(), OneHotEncoder(), OrdinalEncoder(), TargetEncoder()],
    "general_transformer__scoring_encoder" : [BinaryEncoder(), OneHotEncoder(), OrdinalEncoder(), TargetEncoder()],
}

grid_pipeline = pipelineFactory.create_pipeline(train_df,
                                                ModelType.REGRE_BASELINE,
                                                verbose_level=1,
                                                evaluation=EvaluationType.GRID_SEARCH,
                                                param_grid=param_grid,
                                                n_folds=2,
                                                workers=16)

poincare_embedddings_transformer = PoincareEmbedding(graph=graph)
grid_pipeline.add_new_step(poincare_embedddings_transformer, "embeddings_transformer")
#grid_pipeline.add_new_step(ColumnKeeper(columns=["poincare_embedding_dim1", "poincare_embedding_dim2"]), "column_keeper")
grid_pipeline.add_new_step(OpenMLMetaFeatureTransformer(), "dataset_transformer")
grid_pipeline.add_new_step(GeneralPurposeEncoderTransformer(), "general_transformer")
#grid_pipeline.add_new_step(PrintDataframe(verbose=grid_pipeline._verbose_level), "print_df_1")
grid_pipeline.change_estimator(LGBMRegressor())


grid_pipeline.run()


Starting pipeline using method: EvaluationType.GRID_SEARCH
Performing grid search


 11%|█         | 21/192 [07:12<53:44, 18.85s/it]  

'NoneType' object is not iterable


 16%|█▌        | 31/192 [10:36<52:59, 19.75s/it]

'NoneType' object is not iterable


...

The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.



 54%|█████▎    | 103/192 [35:19<28:09, 18.99s/it]

The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.



100%|██████████| 192/192 [1:06:19<00:00, 20.73s/it]


Finished running the pipeline
Evaluation metrics:
    best_score: 0.3499
    best_params: {'embeddings_transformer__epochs': 500, 'embeddings_transformer__batch_size': 50, 'general_transformer__model_encoder': OrdinalEncoder(cols=['model'],
               mapping=[{'col': 'model', 'data_type': dtype('O'),
                         'mapping': LR       1
SVC      2
LGBMC    3
DTC      4
KNC      5
NaN     -2
dtype: int64}]), 'general_transformer__tuning_encoder': BinaryEncoder(cols=['tuning'],
              mapping=[{'col': 'tuning',
                        'mapping':     tuning_0  tuning_1
 1         0         1
 2         1         0
 3         1         1
-1         0         0
-2         0         0}]), 'general_transformer__scoring_encoder': BinaryEncoder(cols=['scoring'],
              mapping=[{'col': 'scoring',
                        'mapping':     scoring_0  scoring_1
 1          0          1
 2          1          0
 3          1          1
-1          0          0
-2          

In [4]:
# imports
import os
import pandas as pd
from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType
from lightgbm import LGBMRegressor
from src.pipeline.pipeline_factory import PipelineFactory
from src.features.encoder_utils import load_graph
from src.pipeline.pipeline_transformers import PoincareEmbedding, ColumnKeeper, OpenMLMetaFeatureTransformer, GeneralPurposeEncoderTransformer
from category_encoders.binary import BinaryEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder


train_df = config.load_traindata_for_regression()
pipelineFactory = PipelineFactory()

graph = load_graph(config.ROOT_DIR / "data/external/graphs/encodings_graph.adjlist")


param_grid = {
    "embeddings_transformer__epochs": [500],
    "embeddings_transformer__batch_size": [50],
    "embeddings_transformer__size": [3, 4, 5],
    "embeddings_transformer__alpha": [0.1],
    "general_transformer__model_encoder" : [OrdinalEncoder()],
    "general_transformer__tuning_encoder" : [BinaryEncoder()],
    "general_transformer__scoring_encoder" : [BinaryEncoder()],
}

grid_pipeline = pipelineFactory.create_pipeline(train_df,
                                                ModelType.REGRE_BASELINE,
                                                verbose_level=1,
                                                evaluation=EvaluationType.GRID_SEARCH,
                                                param_grid=param_grid,
                                                n_folds=2,
                                                workers=16)

poincare_embedddings_transformer = PoincareEmbedding(graph=graph)
grid_pipeline.add_new_step(poincare_embedddings_transformer, "embeddings_transformer")
#grid_pipeline.add_new_step(ColumnKeeper(columns=["poincare_embedding_dim1", "poincare_embedding_dim2"]), "column_keeper")
grid_pipeline.add_new_step(OpenMLMetaFeatureTransformer(), "dataset_transformer")
grid_pipeline.add_new_step(GeneralPurposeEncoderTransformer(), "general_transformer")
#grid_pipeline.add_new_step(PrintDataframe(verbose=grid_pipeline._verbose_level), "print_df_1")
grid_pipeline.change_estimator(LGBMRegressor())


grid_pipeline.run()


Starting pipeline using method: EvaluationType.GRID_SEARCH
Performing grid search


100%|██████████| 3/3 [01:01<00:00, 20.34s/it]

Finished running the pipeline
Evaluation metrics:
    best_score: 0.3345
    best_params: {'embeddings_transformer__epochs': 500, 'embeddings_transformer__batch_size': 50, 'embeddings_transformer__size': 3, 'embeddings_transformer__alpha': 0.1, 'general_transformer__model_encoder': OrdinalEncoder(cols=['model'],
               mapping=[{'col': 'model', 'data_type': dtype('O'),
                         'mapping': LR       1
SVC      2
LGBMC    3
DTC      4
KNC      5
NaN     -2
dtype: int64}]), 'general_transformer__tuning_encoder': BinaryEncoder(cols=['tuning'],
              mapping=[{'col': 'tuning',
                        'mapping':     tuning_0  tuning_1
 1         0         1
 2         1         0
 3         1         1
-1         0         0
-2         0         0}]), 'general_transformer__scoring_encoder': BinaryEncoder(cols=['scoring'],
              mapping=[{'col': 'scoring',
                        'mapping':     scoring_0  scoring_1
 1          0          1
 2          1 

In [2]:
# imports
import os
import pandas as pd
from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType
from lightgbm import LGBMRegressor
from src.pipeline.pipeline_factory import PipelineFactory
from src.features.encoder_utils import load_graph
from src.pipeline.pipeline_transformers import PoincareEmbedding, ColumnKeeper, OpenMLMetaFeatureTransformer, GeneralPurposeEncoderTransformer, Node2VecEmbedding
from category_encoders.binary import BinaryEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder


train_df = config.load_traindata_for_regression()
pipelineFactory = PipelineFactory()

graph = load_graph(config.ROOT_DIR / "data/external/graphs/encodings_graph.adjlist")


param_grid = {
    "embeddings_transformer__dimensions": [2],
    "embeddings_transformer__walk_length": [50],
    # "embeddings_transformer__num_walks": [1000],
    # "embeddings_transformer__workers": [1],
    "general_transformer__model_encoder" : [BinaryEncoder(), OneHotEncoder(), OrdinalEncoder(), TargetEncoder()],
    "general_transformer__tuning_encoder" : [BinaryEncoder(), OneHotEncoder(), OrdinalEncoder(), TargetEncoder()],
    "general_transformer__scoring_encoder" : [BinaryEncoder(), OneHotEncoder(), OrdinalEncoder(), TargetEncoder()],
}

grid_pipeline = pipelineFactory.create_pipeline(train_df,
                                                ModelType.REGRE_BASELINE,
                                                verbose_level=1,
                                                evaluation=EvaluationType.GRID_SEARCH,
                                                param_grid=param_grid,
                                                n_folds=2,
                                                workers=16)

n2v_embedddings_transformer = Node2VecEmbedding(graph=graph)
grid_pipeline.add_new_step(n2v_embedddings_transformer, "embeddings_transformer")
#grid_pipeline.add_new_step(ColumnKeeper(columns=["poincare_embedding_dim1", "poincare_embedding_dim2"]), "column_keeper")
grid_pipeline.add_new_step(OpenMLMetaFeatureTransformer(), "dataset_transformer")
grid_pipeline.add_new_step(GeneralPurposeEncoderTransformer(), "general_transformer")
#grid_pipeline.add_new_step(PrintDataframe(verbose=grid_pipeline._verbose_level), "print_df_1")
grid_pipeline.change_estimator(LGBMRegressor())


grid_pipeline.run()


Starting pipeline using method: EvaluationType.GRID_SEARCH
Performing grid search


  0%|          | 0/64 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/45 [00:00<?, ?it/s]











Generating walks (CPU: 1):  11%|█         | 112/1000 [00:00<00:06, 130.98it/s]

Computing transition probabilities:   0%|          | 0/45 [00:00<?, ?it/s]

...

























































































































Generating walks (CPU: 1): 100%|██████████| 1000/1000 [00:14<00:00, 69.55it/s]


Computing transition probabilities:   0%|          | 0/45 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/45 [00:00<?, ?it/s]




























































































































 77%|███████▋  | 49/64 [42:30<13:00, 52.06s/it]

CTRL+C pressed. Stopping the execution...
Finished running the pipeline
Evaluation metrics:
    best_score: 0.1509
    best_params: {'embeddings_transformer__dimensions': 2, 'embeddings_transformer__walk_length': 50, 'general_transformer__model_encoder': OrdinalEncoder(cols=['model'],
               mapping=[{'col': 'model', 'data_type': dtype('O'),
                         'mapping': LR       1
SVC      2
LGBMC    3
DTC      4
KNC      5
NaN     -2
dtype: int64}]), 'general_transformer__tuning_encoder': TargetEncoder(cols=['tuning']), 'general_transformer__scoring_encoder': OneHotEncoder(cols=['scoring'])}
